<a href="https://colab.research.google.com/github/sresthabugu/Sentimental-and-toxicity/blob/main/hindisen3indi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer
!pip install evaluate
from datasets import load_dataset,DatasetDict
from transformers import AutoTokenizer,AutoModelForSequenceClassification, DataCollatorWithPadding
import evaluate

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hi_3500.ods to hi_3500.ods


In [ ]:
!pip3 install pandas_ods_reader
from pandas_ods_reader import read_ods

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ezodf: filename=ezodf-0.3.2-py2.py3-none-any.whl size=49003 sha256=4b3c06f3f60466b454ddcbbe9e3d080ada7848f6369151f7d2499b3c2ec6d4de
  Stored in directory: /root/.cache/pip/wheels/50/09/62/ea2c44e6ffdd067d6c8d3f557af0734a195bd252d6901f23ca
Successfully built ezodf


In [ ]:
data = read_ods("hi_3500.ods",1, headers=False)
data.head()

,column.0,column.1
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative


In [ ]:
num_entries = len(data)
print(f'The dataset has {num_entries} entries.')


The dataset has 9077 entries.


In [ ]:
data = data.sample(n=1000)


In [ ]:
data['column.1'].unique()

array(['positive', 'neutral', 'negative'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['column.1'] = label_encoder.fit_transform(data['column.1'])

In [ ]:
data['column.1'].unique()

array([2, 1, 0])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['column.0'], data['column.1'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()
df_val = pd.DataFrame()

In [ ]:
df_train['texts'] = X_train
df_train['label'] = y_train
df_test['texts'] = X_test
df_test['label'] = y_test
df_val['texts'] = X_val
df_val['label'] = y_val

In [ ]:
import datasets
from datasets import load_dataset, DatasetDict
utube_sent = DatasetDict()
utube_sent["train"] = datasets.Dataset.from_pandas(df = df_train)
utube_sent["test"] = datasets.Dataset.from_pandas(df = df_test)
utube_sent['val'] = datasets.Dataset.from_pandas(df = df_val)
utube_sent

DatasetDict({
    train: Dataset({
        features: ['texts', 'label', '__index_level_0__'],
        num_rows: 640
    })
    test: Dataset({
        features: ['texts', 'label', '__index_level_0__'],
        num_rows: 200
    })
    val: Dataset({
        features: ['texts', 'label', '__index_level_0__'],
        num_rows: 160
    })
})

In [ ]:
utube_sent = utube_sent.rename_column("label","labels")
utube_sent = utube_sent.rename_column("texts","text")
utube_sent = utube_sent.remove_columns("__index_level_0__")


utube_sent

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 640
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 200
    })
    val: Dataset({
        features: ['text', 'labels'],
        num_rows: 160
    })
})

In [ ]:
checkpoint = 'ai4bharat/indic-bert'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_fn(examples):
    return tokenizer(examples['text'], truncation='longest_first')

In [ ]:
tokenized_datasets = utube_sent.map(tokenize_fn, batched = True)
tokenized_datasets

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 640
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200
    })
    val: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 160
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy"])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
!pip install accelerate -U

In [ ]:
!pip show accelerate

Name: accelerate
Version: 0.33.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
import accelerate
print(accelerate.__version__)


0.32.1


In [ ]:
!pip install --upgrade accelerate

In [ ]:
pip install transformers[torch]


In [ ]:
!pip install wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.6/303.6 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login(key='1ed359577e44aad89b7fdd568b565b0c8be5a426')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install --upgrade accelerate
!pip install --upgrade transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
!pip install datasets transformers


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

training_args = TrainingArguments(
    output_dir="test-trainer",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=15,
    learning_rate=2e-5,
    weight_decay=0.1
)





/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer =Trainer(model,training_args,train_dataset=tokenized_datasets['train'],eval_dataset=tokenized_datasets['val'], data_collator=data_collator, tokenizer=tokenizer)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: tarik2568 (tarikul_tamiti). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.964682
2,No log,0.741799
3,No log,0.784715
4,No log,0.727779
5,No log,0.724226
6,No log,0.770428
7,No log,0.726645
8,No log,0.717864
9,No log,0.819374
10,No log,0.759094


TrainOutput(global_step=600, training_loss=0.44681992530822756, metrics={'train_runtime': 153.4481, 'train_samples_per_second': 62.562, 'train_steps_per_second': 3.91, 'total_flos': 24251502194880.0, 'train_loss': 0.44681992530822756, 'epoch': 15.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets["val"])
ground_truth_labels = predictions.label_ids
predicted_labels = predictions.predictions.argmax(axis=1)  # Extract predicted labels

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

accuracy = accuracy_score(ground_truth_labels, predicted_labels)
f1_macro = f1_score(ground_truth_labels, predicted_labels, average='macro')
f1_micro = f1_score(ground_truth_labels, predicted_labels, average='micro')

precision = precision_score(ground_truth_labels, predicted_labels, average="weighted")
recall = recall_score(ground_truth_labels, predicted_labels, average="weighted")
class_report = classification_report(ground_truth_labels, predicted_labels)
conf_matrix = confusion_matrix(ground_truth_labels, predicted_labels)

print("Accuracy:", accuracy)
print("F1 Macro:", f1_macro)
print("F1 Micro:", f1_micro)
print('Precision:', precision)
print("Recall:", recall)
print("Classification Report:\n", class_report)
print("Confusion Matrix: \n", conf_matrix)

Accuracy: 0.66875
F1 Macro: 0.6815013010786091
F1 Micro: 0.66875
Precision: 0.6637294917967187
Recall: 0.66875
Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.61      0.59        56
           1       0.84      0.93      0.88        44
           2       0.63      0.53      0.58        60

    accuracy                           0.67       160
   macro avg       0.68      0.69      0.68       160
weighted avg       0.66      0.67      0.66       160

Confusion Matrix: 
 [[34  4 18]
 [ 2 41  1]
 [24  4 32]]
